In [0]:
from bs4 import BeautifulSoup
import requests
from decimal import Decimal

import re

import sqlite3
from google.colab import drive
drive.mount('/content/gdrive')
path = "/content/gdrive/My Drive/Datathon/Data/articles.db"
conn = sqlite3.connect(path)
c = conn.cursor()

c.execute('PRAGMA foreign_keys = ON')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
# Create companies table
create_articles_table_command = '''
CREATE TABLE IF NOT EXISTS articles (
    title varchar(255) NOT NULL,
    day INT NOT NULL,
    month INT NOT NULL,
    year INT NOT NULL,
    PRIMARY KEY(title, day, month, year)
    );
'''
c.execute(create_articles_table_command)
conn.commit()

# Create modified table
create_modified_table_command = '''
CREATE TABLE IF NOT EXISTS cleaned_articles (
    title varchar(255) NOT NULL,
    date DATETIME NOT NULL,
    PRIMARY KEY(title, date)
    );
'''
c.execute(create_modified_table_command)
conn.commit()

In [0]:
SEARCH_URL="http://data.people.com.cn/rmrb/s?qs=%7B%22cds%22%3A%5B%7B%22cdr%22%3A%22AND%22%2C%22cds%22%3A%5B%7B%22fld%22%3A%22title%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E7%8C%AA%E6%B5%81%E6%84%9F%22%7D%2C%7B%22fld%22%3A%22subTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E7%8C%AA%E6%B5%81%E6%84%9F%22%7D%2C%7B%22fld%22%3A%22introTitle%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22false%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E7%8C%AA%E6%B5%81%E6%84%9F%22%7D%2C%7B%22fld%22%3A%22contentText%22%2C%22cdr%22%3A%22OR%22%2C%22hlt%22%3A%22true%22%2C%22vlr%22%3A%22OR%22%2C%22val%22%3A%22%E7%8C%AA%E6%B5%81%E6%84%9F%22%7D%5D%7D%5D%2C%22obs%22%3A%5B%7B%22fld%22%3A%22dataTime%22%2C%22drt%22%3A%22DESC%22%7D%5D%7D&tr=A&ss=1&pageNo=1&pageSize=1000"



headers = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) ' 
                      'AppleWebKit/537.11 (KHTML, like Gecko) '
                      'Chrome/23.0.1271.64 Safari/537.11',
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
        'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
        'Accept-Encoding': 'none',
        'Accept-Language': 'en-US,en;q=0.8',
        'Connection': 'keep-alive'}

# Use BeautifulSoup and requests to collect data required for the assignment.
req = requests.get(SEARCH_URL, headers=headers)
html_dump = BeautifulSoup(req.text, 'html.parser')


In [0]:
items = html_dump.find_all('div', {'class':'sreach_li'})
print(len(items))

for item in items:
  title = item.find('a').text
  date_and_issue = item.find('div', {'class':'listinfo'}).text.split(' ', 1)[0].strip('\t\n')

  date = re.findall(r'(\d+)', date_and_issue)

  year = date[0]
  month = date[1]
  day = date[2]
  c.execute(
        '''INSERT OR IGNORE INTO articles VALUES (?, ?, ?, ?);''', 
        (title, day, month, year))
  
conn.commit()



2004


In [0]:
c.execute('''SELECT COUNT(*) FROM articles''')

for row in c:
    print(row)

(2583,)


In [0]:
c.execute('''SELECT month, count(month) FROM articles WHERE year == 2013 GROUP BY month ORDER BY month ASC''')

for row in c:
    print(row)

(1, 5)
(2, 4)
(3, 2)
(4, 78)
(5, 21)
(6, 6)
(7, 7)
(8, 7)
(9, 2)
(10, 2)
(11, 5)
(12, 3)


In [0]:
c.execute('''SELECT month, count(month) FROM articles WHERE year == 2009 GROUP BY month ORDER BY month ASC''')

for row in c:
    print(row)

(1, 17)
(2, 13)
(4, 10)
(5, 170)
(6, 82)
(7, 59)
(8, 36)
(9, 38)
(10, 35)
(11, 59)
(12, 25)


In [0]:
c.execute('''SELECT year, month, count(month) FROM articles WHERE (year == 2005 AND month >= 9) OR (year == 2006 AND month <= 8) GROUP BY month ORDER BY year ASC, month ASC''')

for row in c:
    print(row)

(2005, 9, 14)
(2005, 10, 39)
(2005, 11, 162)
(2005, 12, 100)
(2006, 1, 67)
(2006, 2, 45)
(2006, 3, 53)
(2006, 4, 29)
(2006, 5, 16)
(2006, 6, 14)
(2006, 7, 19)
(2006, 8, 17)


In [0]:
c.execute('''SELECT day, month, year, title FROM articles WHERE year == 2013''')

results = c

for row in results:
    title = row[3]
    date = str(row[2])+'-'+str(row[1])+'-'+str(row[0])
    print(title)
    c.execute(
        '''INSERT OR IGNORE INTO cleaned_articles VALUES (?, ?);''', 
        (title, date))
  
conn.commit()

战斗在一线的“流感侦探”(人物)


In [65]:
c.execute('''SELECT COUNT(*) FROM articles''')

for row in c:
    print(row)

(2583,)
